In [0]:
import pandas as pd
import numpy
df = pd.read_csv('BX-Book-Ratings.csv', names=["User-ID", "ISBN", "Book-Rating"], delimiter=";")
rating_matrix = numpy.ndarray(shape=(50000,50000), dtype = 'int64')
df_mat = df.as_matrix()[1:]
#print(df_mat)

C:\Users\karth\Anaconda2\envs\cs670\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
%store df_mat

Stored 'df_mat' (ndarray)


In [0]:
%store -r df_mat
totalratings = len(df_mat[:,0])
print totalratings

1149780


In [0]:
import numpy as np
df_mat[:,0] = df_mat[:,0].astype('int64')
df_mat[:,2] = df_mat[:,2].astype('int64')

usercount = (len(np.unique(df_mat[:,0])))
bookcount = (len(np.unique(df_mat[:,1])))
#numpy.unique(df_mat[:,1]).shape

In [0]:
print usercount
print bookcount

105283
340556


In [0]:
bookid = 0
usernum = 0
bookmap = {}
usermap = {}
useridslist = []
for k in range(0,800000):
    userid = df_mat[k,0]
    isbn = df_mat[k,1]
    if isbn not in bookmap:
        if bookid < 50000:
            bookmap[isbn] = bookid
            if userid not in usermap:
                if usernum < 50000:
                    usermap[userid]=usernum
                    rating_matrix[usernum, bookid] = df_mat[k,2]
                    usernum+=1
            else:
                rating_matrix[usermap[userid], bookid] = df_mat[k,2]
            bookid = bookid+1
    else:
        if userid not in usermap:
            if usernum < 50000:
                usermap[userid]=usernum
                rating_matrix[usernum, bookmap[isbn]] = df_mat[k,2]
                usernum+=1
        else:
            rating_matrix[usermap[userid], bookmap[isbn]] = df_mat[k,2]
            


In [0]:
print(rating_matrix.shape)

(50000L, 50000L)


In [0]:
import numpy as np
user_list = np.unique(df_mat[:,0])
print len(user_list)

105283


In [0]:
print len(bookmap.values())
print len(usermap.values())

#print df_mat[0,0]
#print bookmap[df_mat[0,1]]

50000
50000


In [0]:
import networkx as nx
from networkx.algorithms import bipartite

In [0]:
# constructing a bipartite graph

B = nx.Graph()
B.add_nodes_from(usermap.values(), bipartite=0) #0 indicates one class of nodes, here: users
B.add_nodes_from(bookmap.values(), bipartite=1) #1 for isbn

In [0]:
edge_ls = []

for k in range(0,800000):
    ls = []
    if k%50000==0:
        print k
    if df_mat[k,0] in usermap.keys() and df_mat[k,1] in bookmap.keys() and df_mat[k,2]!=0:
        ls.append(usermap[df_mat[k,0]])
        ls.append(bookmap[df_mat[k,1]])
        if df_mat[k,2] > 6:
            rate = 'L' # for love
        else: rate = 'H' # for hate
        ls.append(rate)
        edge_ls.append(ls)
#print edge_ls
B.add_weighted_edges_from(edge_ls)

'''
ls.append()
'''

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000


'\nls.append()\n'

In [0]:
print(len(B))

50000


In [0]:
# import networkx as nx
# from networkx.algorithms import bipartite

# B = nx.Graph()
# B.add_nodes_from(usermap.values(), bipartite=0)
# B.add_nodes_from(bookmap.values(), bipartite=1)

# edge_ls = []
# for i in range(0,50000):
#     for j in range(0,50000):
#         if rating_matrix[i,j] !=0:
#             ls = []
#             ls.append(i)
#             ls.append(j)
#             if rating_matrix[i,j] > 6:
#                 rate = 'L' # for love
#             else: 
#                 rate = 'H' # for hate
#             ls.append(rate)
#             edge_ls.append(ls)
# B.add_weighted_edges_from(edge_ls)

In [0]:
%store -r B
import pickle
pickle.dump(B,open("a.p","wb"))

In [0]:
import pickle
#B= pickle.load(open("a.p","rb"))
from collections import defaultdict
Dictionary = defaultdict(dict)
count =0
dictionary_adj = dict(B.adjacency()) 
for user1 in usermap.values():
    BFS_Dict= dict(nx.bfs_successors(B,user1))
    User_item_list = []
    User_item_list.append(user1)
    edge_list=[]
    if count%10000 ==0:
        print "Reached user ",count+1
    count+=1
    if bool(BFS_Dict):
        for book1 in BFS_Dict[user1]:
            if book1 in BFS_Dict.keys():
                edge_list.append(dictionary_adj[user1][book1]['weight'])
                for user2 in BFS_Dict[book1]:
                    edge_list.append(dictionary_adj[book1][user2]['weight'])    
                    '''
                    if count%100 ==0:
                        print "Hello"
                        print dictionary_adj[user2]
                        print BFS_Dict[user1]
                    '''
                    for book2 in dictionary_adj[user2].keys(): 
                        if book2 in BFS_Dict[user1] and book2!=book1:
                            edge_list.append(dictionary_adj[user2][book2]['weight'])
                            User_item_list.append(book2)
                            Dictionary[tuple(User_item_list)][tuple(edge_list)] = Dictionary[tuple(User_item_list)].get(tuple(edge_list),0)+1
                            Dictionary[tuple(User_item_list)]["Rating"]=rating_matrix[User_item_list[0]][User_item_list[1]]
                            edge_list.pop()
                            User_item_list.pop()
                    edge_list.pop()
                edge_list.pop()
            
#print(Dictionary)
                        

Reached user  1
Reached user  10001
Reached user  20001
Reached user  30001
Reached user  40001


In [0]:
%store Dictionary
import pickle
pickle.dump(Dictionary,open("b.p","wb"))

Stored 'Dictionary' (defaultdict)


In [0]:
# naming all configurations
con = []

con.append([('H', 'H', 'H')])
con.append([('H', 'H', 'L')])
con.append([('H', 'L', 'L')])
con.append([('L', 'L', 'L')])
con.append([('L', 'L', 'H')])
con.append([('L', 'H', 'H')])
con.append([('H', 'L', 'H')])
con.append([('L', 'L', 'L')])



In [0]:
%store -r Dictionary
import numpy as np
config_mat = np.zeros((len(Dictionary), 9))
pair_dic = {}
i=0
for k in Dictionary.keys():
    userItemConfig = Dictionary[k]
    pair_dic[i] = k
    for key in userItemConfig.keys():
        for j in range(0,8):
            if con[j][0] == key:
                config_mat[i,j] = userItemConfig[key]
    if rating_matrix[k[0],k[1]]!=0:
        if rating_matrix[k[0],k[1]] > 6:
            config_mat[i,-1] = 1
        else: config_mat[i,-1] = 0
    #config_mat[i,-1] = rating_matrix[k[0],k[1]] 
    i+=1
            

In [0]:
%store config_mat
import pickle
pickle.dump(config_mat,open("c.p","wb"))

Stored 'config_mat' (ndarray)


In [0]:
%store -r config_mat
print(config_mat)

[[ 0.  0.  0. ...  0.  1.  1.]
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  1.  0.]
 ...
 [ 0.  1. 10. ...  1.  8.  0.]
 [ 0.  0.  0. ...  1.  3.  0.]
 [ 0.  1.  2. ...  0.  0.  0.]]


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(config_mat[:,0:8], config_mat[:,-1], test_size=0.30, shuffle=False)

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
mnb_y_pred = mnb.predict(X_test)
print len(mnb_y_pred)

48388


In [0]:
#print len(y_test)

from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, mnb_y_pred))
print "RMSE with Naive Bayes:",rmse

RMSE with Naive Bayes: 0.66533580754


In [0]:
%store X_train
%store X_test
%store y_train
%store y_test

Stored 'X_train' (ndarray)
Stored 'X_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)


In [0]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
KNN_y_pred = knn.predict(X_test)
rmse_knn = sqrt(mean_squared_error(y_test, KNN_y_pred))
print "RMSE with kNN:", rmse_knn

RMSE with kNN: 0.665118341998


In [0]:
%store df_mat_book
%store bookmap
%store usermap
%store pair_dic
%store lr_y_pred

Stored 'df_mat_book' (ndarray)
Stored 'bookmap' (dict)
Stored 'usermap' (dict)
Stored 'pair_dic' (dict)
Stored 'lr_y_pred' (ndarray)


 USER ID:  146230  BOOK NAME:  Harry Potter and the Chamber of Secrets (Book 2)  PREDICTED:  Hate  ACTUAL:  Love
 USER ID:  51399  BOOK NAME:  The Sophisticated traveler: Winter, love it or leave it  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  96357  BOOK NAME:  The Bonfire of the Vanities  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  132203  BOOK NAME:  Skeleton Crew  PREDICTED:  Hate  ACTUAL:  Love
 USER ID:  85510  BOOK NAME:  Eyes of Prey  PREDICTED:  Hate  ACTUAL:  Love
Stored 'book_name' (list)
Stored 'isbn' (list)


In [16]:
#print Bookcount
counter = 0
list_book =[]
for key,item in sorted(Bookcount.iteritems(),key=lambda (k,v):(v,k),reverse=True):
    if counter == 20:
        break
    counter+=1
    list_book.append(key)
    print key,item
print list_book
%store list_book

Selected Poems 27
Little Women 24
Wuthering Heights 21
The Secret Garden 20
Dracula 20
Adventures of Huckleberry Finn 20
Jane Eyre 19
The Night Before Christmas 18
Pride and Prejudice 18
Great Expectations 17
Masquerade 16
Frankenstein 16
Black Beauty 16
The Gift 15
Emma 15
Beloved 15
Nemesis 14
The Wedding 13
The Secret 13
Robinson Crusoe 13
['Selected Poems', 'Little Women', 'Wuthering Heights', 'The Secret Garden', 'Dracula', 'Adventures of Huckleberry Finn', 'Jane Eyre', 'The Night Before Christmas', 'Pride and Prejudice', 'Great Expectations', 'Masquerade', 'Frankenstein', 'Black Beauty', 'The Gift', 'Emma', 'Beloved', 'Nemesis', 'The Wedding', 'The Secret', 'Robinson Crusoe']


In [22]:
%store -r Dictionary
counter=0
from collections import defaultdict
Dict_new = defaultdict(dict)
for key, item in Dictionary.iteritems():
    val = bookmap.keys()[bookmap.values().index(key[1])]
    if val in isbn:
        bookname = book_name[isbn.index(val)]
        if bookname in list_book:
            Dict_new[bookname][usermap.keys()[usermap.values().index(key[0])]]=item
for key,item in Dict_new.iteritems():
    print key,item

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


In [24]:
print (len(Dict_new))

18


In [23]:
for key,item in Dict_new.iteritems():
    print key,item

Jane Eyre {62881L: {'Rating': 0, ('L', 'L', 'H'): 1}, 72451L: {'Rating': 0, ('H', 'L', 'L'): 1, ('H', 'H', 'L'): 1}, 21576L: {'Rating': 10, ('L', 'L', 'L'): 1}, 41097L: {'Rating': 0, ('L', 'H', 'L'): 1}, 35980L: {'Rating': 0, ('L', 'H', 'L'): 1}, 91373L: {'Rating': 0, ('L', 'L', 'L'): 1}, 8370L: {'Rating': 8, ('L', 'L', 'L'): 1}, 11676L: {'Rating': 8, ('L', 'L', 'L'): 3}, 171295L: {'Rating': 8, ('L', 'L', 'L'): 1}}
Nemesis {40640L: {'Rating': 5, ('L', 'H', 'L'): 1}, 68533L: {'Rating': 0, ('H', 'L', 'L'): 1}, 145109L: {'Rating': 9, ('L', 'L', 'H'): 1}, 1166L: {'Rating': 0, ('L', 'H', 'H'): 1}, 154975L: {'Rating': 8, ('H', 'H', 'L'): 1}}
Emma {30081L: {'Rating': 10, ('L', 'L', 'L'): 2}, 94307L: {'Rating': 10, ('H', 'L', 'L'): 1}, 111190L: {'Rating': 9, ('L', 'L', 'L'): 1}, 127233L: {'Rating': 9, ('H', 'L', 'L'): 1, ('L', 'H', 'L'): 1}, 1131L: {'Rating': 8, ('L', 'L', 'L'): 1}, 8332L: {'Rating': 0, ('H', 'L', 'L'): 1}, 21618L: {'Rating': 9, ('L', 'H', 'L'): 1}, 115572L: {'Rating': 9, ('L'

In [0]:
# import pandas as pd
# df_book = pd.read_csv('BX-Books.csv', names=["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"], delimiter=";")
# df_mat_book = df_book.as_matrix()[1:]

In [0]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error
from math import sqrt

clf = LinearSVC()

clf = clf.fit(X_train, y_train)
svc_y_pred = clf.predict(X_test)
rmse_svc = sqrt(mean_squared_error(y_test, svc_y_pred))
print "RMSE with SVM:", rmse_svc

RMSE with SVM: 0.629274668368


In [0]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import tree
from sklearn.metrics import mean_squared_error
from math import sqrt

dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
rmse_dt = sqrt(mean_squared_error(y_test, dt_y_pred))
print "RMSE with Decision tree:", rmse_dt

RMSE with Decision tree: 0.633529668826


In [0]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt

lr = linear_model.LogisticRegression()
lr = lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
rmse_lr = sqrt(mean_squared_error(y_test, lr_y_pred))
print "RMSE with Logistic Regression:", rmse_lr

RMSE with Logistic Regression: 0.628995454256


In [26]:
# Demo
%store -r X_train
%store -r y_test
%store -r lr_y_pred
%store -r df_mat_book
%store -r usermap
%store -r bookmap
%store -r pair_dic


from random import randint
isbn =[row[0] for row in df_mat_book]
book_name =[row[1] for row in df_mat_book]
count=0
# 5922 , 23938 
offset_train = len(X_train)
offset_test = len(y_test)
while(count<5):
    rnd = randint(0,offset_test-1)
    #print pair_dic[rnd][0]
    if lr_y_pred[rnd] == 0:
        lr_p = 'Hate'
    else : lr_p = 'Love'
    if y_test[rnd] == 0:
        yt_p = 'Hate'
    else : yt_p = 'Love'   
    if isbn.count(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])!=0:
        print " USER ID: ",usermap.keys()[usermap.values().index(pair_dic[rnd+offset_train][0])],
        print " BOOK NAME: ",book_name[isbn.index(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])],
        print " PREDICTED: ",lr_p," ACTUAL: ",yt_p
        count+=1
%store book_name
%store isbn

 USER ID:  31846  BOOK NAME:  Innocent Blood  PREDICTED:  Love  ACTUAL:  Hate
 USER ID:  4402  BOOK NAME:  Lightning  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  243  BOOK NAME:  The God of Small Things  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  50215  BOOK NAME:  The Love of Stones: A Novel  PREDICTED:  Hate  ACTUAL:  Love
 USER ID:  83869  BOOK NAME:  Managing Your Mind: The Mental Fitness Guide  PREDICTED:  Hate  ACTUAL:  Love
Stored 'book_name' (list)
Stored 'isbn' (list)


In [31]:
%store -r book_name
%store -r isbn
%store -r Dictionary
Bookcount = {}
for i in range(len(book_name)):
    if isbn[i] in bookmap.keys():
        if book_name[i] not in Bookcount.keys():
            Bookcount[book_name[i]]=book_name.count(book_name[i])  
counter = 0
top_books =[]
print ("Top Books")
for key,item in sorted(Bookcount.iteritems(),key=lambda (k,v):(v,k),reverse=True):
    if counter == 20:
        break
    counter+=1
    top_books.append(key)
    print key,item
counter=0
print("Final Dicitonary")
from collections import defaultdict
Dict_output = defaultdict(dict)
for key, item in Dictionary.iteritems():
    isbn_temp = bookmap.keys()[bookmap.values().index(key[1])]
    if isbn_temp in isbn:
        bookname = book_name[isbn.index(isbn_temp)]
        if bookname in top_books:
            Dict_output["Bookname :" + str(bookname)]["User id : " + str(usermap.keys()[usermap.values().index(key[0])])]=item
for key,item in Dict_output.iteritems():
    print key,item

Top Books
Selected Poems 27
Little Women 24
Wuthering Heights 21
The Secret Garden 20
Dracula 20
Adventures of Huckleberry Finn 20
Jane Eyre 19
The Night Before Christmas 18
Pride and Prejudice 18
Great Expectations 17
Masquerade 16
Frankenstein 16
Black Beauty 16
The Gift 15
Emma 15
Beloved 15
Nemesis 14
The Wedding 13
The Secret 13
Robinson Crusoe 13
Final Dicitonary
Bookname :Jane Eyre {'User id : 35980': {'Rating': 0, ('L', 'H', 'L'): 1}, 'User id : 62881': {'Rating': 0, ('L', 'L', 'H'): 1}, 'User id : 91373': {'Rating': 0, ('L', 'L', 'L'): 1}, 'User id : 11676': {'Rating': 8, ('L', 'L', 'L'): 3}, 'User id : 8370': {'Rating': 8, ('L', 'L', 'L'): 1}, 'User id : 41097': {'Rating': 0, ('L', 'H', 'L'): 1}, 'User id : 21576': {'Rating': 10, ('L', 'L', 'L'): 1}, 'User id : 72451': {'Rating': 0, ('H', 'L', 'L'): 1, ('H', 'H', 'L'): 1}, 'User id : 171295': {'Rating': 8, ('L', 'L', 'L'): 1}}
Bookname :The Secret {'User id : 12982': {'Rating': 9, ('L', 'L', 'H'): 1, ('H', 'L', 'L'): 1, ('L',